In [18]:
import pandas as pd
import xgboost
from sklearn.model_selection import train_test_split
import random

In [33]:
df = pd.read_csv('./../creditRisk_creditCardFraud/data_fraude.csv')

In [34]:
df = df.rename(columns={'0.1':'Class'}).\
drop('Unnamed: 0',axis=1)

yvar = df.Class
xvars = df[[x for x in df.columns if x != 'Class']]

In [39]:
xtrain, xtest, ytrain, ytest = train_test_split(xvars, yvar, train_size = 0.80, random_state = 2)


In [57]:
trainXGB = xgboost.DMatrix(data=xtrain,
                label=ytrain)


testXGB = xgboost.DMatrix(data=xtest,
                          label=ytest
                         )

In [75]:
best_metric = 0
best_params = list()
iters = 1
for iteration in range(iters):

    params = {
            'tree_method' : 'exact',
            'booster' : 'gbtree', # 'gblinear'
            'eta' : random.uniform(0.01, 0.3),
            'max_depth' : random.randint(5,14),
            'reg_lambda' : random.uniform(0.01, 0.4),
            'reg_alpha' : random.uniform(0.01, 0.4),
            'gamma' : random.randint(0, 20),
            'subsample' : random.uniform(0.5, 1),
            'colsample_bytree' : random.uniform(0.5, 1),
            'objective' : 'binary:logistic',
            'eval_metric' : 'auc'
        }

    xgb_cv = xgboost.cv(
        params = params, 
        dtrain = trainXGB, 
           nfold=10,
           metrics={'auc'}, 
           seed=0,
           callbacks=[xgboost.callback.EvaluationMonitor(show_stdv=True),
                      xgboost.callback.EarlyStopping(2)])
    

[0]	train-auc:0.96948+0.00139	test-auc:0.96561+0.01069
[1]	train-auc:0.97373+0.00279	test-auc:0.97055+0.01067
[2]	train-auc:0.97764+0.00351	test-auc:0.97346+0.01248
[3]	train-auc:0.97867+0.00335	test-auc:0.97485+0.01337
[4]	train-auc:0.98048+0.00272	test-auc:0.97691+0.01246
[5]	train-auc:0.98165+0.00334	test-auc:0.97701+0.01291
[6]	train-auc:0.98290+0.00355	test-auc:0.97813+0.01160
[7]	train-auc:0.98329+0.00392	test-auc:0.97814+0.01175
[8]	train-auc:0.98386+0.00384	test-auc:0.97843+0.01183
[9]	train-auc:0.98425+0.00366	test-auc:0.97873+0.01233


In [ ]:
best_metric = 0
best_params = list()
    
for iteration in 1:iter:

    

    xgb_cv <- 
    xgb.cv(
      nfold = 10,
      params = params,
      nrounds = 20000,
      data = trainXGB,
      verbose = 2,
      maximize = T,
      early_stopping_rounds = 2
    )

auc = xgb_cv$evaluation_log$test_auc_mean[xgb_cv$best_iteration]

if(auc > best_metric){
  best_metric = xgb_cv$evaluation_log$test_auc_mean[xgb_cv$best_iteration]
  best_params = params
}
}

finalXGB <-
xgb.train(
  params = best_params,
  nrounds = 20000,
  data = trainXGB,
  verbose = 2,
  maximize = F,
  early_stopping_rounds = 5,
  watchlist = list(train = trainXGB,
                   test = testXGB)
)

return(list(finalXGB = finalXGB,
          best_params = best_params))
}

}






